# Извлечение данных с помощью Gemma 1 2b

## Загрузка и установка Gemma из Kaggle

In [ ]:
!pip install -q -U tf-keras
!pip install -q -U keras-nlp==0.10.0
!pip install -q -U kagglehub>=0.2.4
!pip install -q -U keras>=3

In [2]:
import os
from google.colab import userdata

os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')

os.environ["KERAS_BACKEND"] = "jax"
#Avoid memory fragmentation on JAX backend.
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

In [3]:
import keras_nlp
import keras
import csv

print("KerasNLP version: ", keras_nlp.__version__)
print("Keras version: ", keras.__version__)

KerasNLP version:  0.10.0
Keras version:  3.9.2


In [ ]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

## Подготовка датасета

In [ ]:
!mkdir "dataset"
!mv niikm_data.zip dataset
%cd dataset
!unzip niikm_data.zip
!rm niikm_data.zip
%cd ..

In [ ]:
DATASET_FOLDER = "dataset/"
dataset_file_names = os.listdir(DATASET_FOLDER)

dataset = []
loaded_file_count = 0
ignored_file_count = 0

print("Загрузка датасета...")
for i in dataset_file_names:
    f = open(DATASET_FOLDER + i, 'r', encoding="utf-8")

    #Файлы, не содержащие информацию о ГОСТ или другом нормативном документе, игнорируются.
    buffer = f.readlines()
    ignore = True
    for j in buffer:
        if j.find("ГОСТ") != -1:
            ignore = False
            break
    if ignore:
        print("    INFO: Файл \"" + i + "\" не содержит информацию о ГОСТ. Файл пропущен.")
        ignored_file_count += 1
    else:
        loaded_file_count += 1
        buffer2 = ""
        for j in buffer:
            buffer2 += j
        dataset.append(buffer2)

    f.close()

print("Готово (" + str(len(dataset_file_names)) + " всего, " + str(loaded_file_count) + " загружено, " + str(ignored_file_count) + " пропущено).")

## Генерация данных

In [12]:
template = """Ты - ИИ-помощник, созданный для поиска информации о свойствах газа в тексте и её преобразовании в формат JSON.
Оформи ответ на вопрос в соответствии с примером. В ответе должно быть:
- название основного газа в составе;
- название газа;
- химическую формулу газа;
- ГОСТ, задающий требования к качеству газа.

ПРИМЕР ВОПРОСА:
НАЗВАНИЕ: Азот жидкий особой чистоты 1 сорт
ГОСТ / НОРМАТИВНЫЙ ДОКУМЕНТ: ГОСТ 9293 - 74
ТРЕБОВАНИЯ К ПРОДУКТУ ПО ГОСТ:
        Наименование показателя                Норма по ГОСТ
        Объёмная доля азота, % не менее                99,999
        Объёмная доля кислорода, % не более                0,0005
        Содержание масла, механических примесей и влаги                выдерживает испытание
        Объёмная доля водорода, % не более                0,0002
        Объёмная доля суммы углеродсодержащих соединений в пересчете на СН4, % не более                0,0003
ОСНОВНЫЕ СВОЙСТВА:
Латинское название:  Nitrogenium
CAS номер: 7727-37-9
UN газа: 1066
UN жидкости: 1977
Код ООН: 1006
Физико-химические свойства:
Электронная конфигурация: 2s22p3
Молекулярая масса: 28.0134
Степень окисления: от +5 до -3
Плотность: 1,251
Удельная теплоёмкость: 1,034
Теплопроводность: 0,026
t кипения: -195,8 °C
t плавления: -210 °C
Внешние признаки: Без цвета вкуса и запаха, химически весьма инертен

ПРИМЕР ОТВЕТА: {"based_on":"на основе Азота","gas_name":"Азот жидкий особой чистоты 1 сорт","gas_letter":"Ar","state_standard":"ГОСТ 9293 - 74"}."""

prompt = template + "\n\nВОПРОС: " + dataset[0] + "\nОТВЕТ:"

generated = gemma_lm.generate(prompt, max_length=1100)
print(generated)

Ты - ИИ-помощник, созданный для поиска информации о свойствах газа в тексте и её преобразовании в формат JSON.
Оформи ответ на вопрос в соответствии с примером. В ответе должно быть:
- название основного газа в составе;
- название газа;
- химическую формулу газа;
- ГОСТ, задающий требования к качеству газа.

ПРИМЕР ВОПРОСА:
НАЗВАНИЕ: Азот жидкий особой чистоты 1 сорт
ГОСТ / НОРМАТИВНЫЙ ДОКУМЕНТ: ГОСТ 9293 - 74
ТРЕБОВАНИЯ К ПРОДУКТУ ПО ГОСТ:
        Наименование показателя                Норма по ГОСТ
        Объёмная доля азота, % не менее                99,999
        Объёмная доля кислорода, % не более                0,0005
        Содержание масла, механических примесей и влаги                выдерживает испытание
        Объёмная доля водорода, % не более                0,0002
        Объёмная доля суммы углеродсодержащих соединений в пересчете на СН4, % не более                0,0003
ОСНОВНЫЕ СВОЙСТВА:
Латинское название:  Nitrogenium
CAS номер: 7727-37-9
UN газа: 1066
UN жидкости

In [13]:
first_position = generated.find("{", generated.find("{") + 1)
last_position = generated.rfind("}")

print(generated[first_position:last_position + 1])

{"based_on":"на основе водорода","gas_name":"Водород газообразный особо чистый марки В","gas_letter":"H","state_standard":"ТУ 2114-016-78538315-2008"}


In [16]:
#Ты - ИИ-помощник, созданный для поиска информации о составе газа в тексте.
#Извлеки данные о составе газа из таблицы требований к продукту по ГОСТ.
#Представь извлечённые данные об объёмных долях в виде списка.

#You are an AI assistant designed to search for the gas composition in the text.
#Extract the info on the gas composition from the text and format it as a list. Each
#item must contain the name of the component, the chemical formula of the component and the percentage share of the component.

template = """Ты - ИИ-помощник, созданный для поиска информации о составе газа в тексте.
Оформи ответ на вопрос в соответствии с примером. Каждый пункт ответа должен включать:
- название компонента газа из вопроса;
- химическую формулу компонента газа из вопроса;
- объёмную долю компонента газа из вопроса.

ПРИМЕР ВОПРОСА:
НАЗВАНИЕ: Азот жидкий особой чистоты 1 сорт
ГОСТ / НОРМАТИВНЫЙ ДОКУМЕНТ: ГОСТ 9293 - 74
ТРЕБОВАНИЯ К ПРОДУКТУ ПО ГОСТ:
        Наименование показателя                Норма по ГОСТ
        Объёмная доля азота, % не менее                99,999
        Объёмная доля кислорода, % не более                0,0005
        Содержание масла, механических примесей и влаги                выдерживает испытание
        Объёмная доля водорода, % не более                0,0002
        Объёмная доля суммы углеродсодержащих соединений в пересчете на СН4, % не более                0,0003
ОСНОВНЫЕ СВОЙСТВА:
Латинское название:  Nitrogenium
CAS номер: 7727-37-9
UN газа: 1066
UN жидкости: 1977
Код ООН: 1006
Физико-химические свойства:
Электронная конфигурация: 2s22p3
Молекулярая масса: 28.0134
Степень окисления: от +5 до -3
Плотность: 1,251
Удельная теплоёмкость: 1,034
Теплопроводность: 0,026
t кипения: -195,8 °C
t плавления: -210 °C
Внешние признаки: Без цвета вкуса и запаха, химически весьма инертен

ПРИМЕР ОТВЕТА:
* Азот - N - не менее 99.999%
* Кислород - O - не более 0.0005%
* Водород - H - не более 0.0002%"""

prompt = template + "\n\nВОПРОС: \n" + dataset[19] + "\nОТВЕТ: "

#Периодически gemma начинает галлюцинировать. При некоторых вводах она упорно лепит
#в ответе информацию про оксид азота, даже если в тексте о нём нет ни слова.
#Проблемы также вызывает непоследовательность в формате таблиц.
generated = gemma_lm.generate(prompt, max_length=1090)
print(generated)

Ты - ИИ-помощник, созданный для поиска информации о составе газа в тексте.
Оформи ответ на вопрос в соответствии с примером. Каждый пункт ответа должен включать:
- название компонента газа;
- химическую формулу компонента газа;
- объёмную долю компонента газа.

ПРИМЕР ВОПРОСА:
НАЗВАНИЕ: Азот жидкий особой чистоты 1 сорт
ГОСТ / НОРМАТИВНЫЙ ДОКУМЕНТ: ГОСТ 9293 - 74
ТРЕБОВАНИЯ К ПРОДУКТУ ПО ГОСТ:
        Наименование показателя                Норма по ГОСТ
        Объёмная доля азота, % не менее                99,999
        Объёмная доля кислорода, % не более                0,0005
        Содержание масла, механических примесей и влаги                выдерживает испытание
        Объёмная доля водорода, % не более                0,0002
        Объёмная доля суммы углеродсодержащих соединений в пересчете на СН4, % не более                0,0003
ОСНОВНЫЕ СВОЙСТВА:
Латинское название:  Nitrogenium
CAS номер: 7727-37-9
UN газа: 1066
UN жидкости: 1977
Код ООН: 1006
Физико-химические свойства:

# Установка Gemma.cpp (веса с Kaggle, остальное с Github)

In [ ]:
!git clone https://github.com/google/gemma.cpp

In [ ]:
%cd /content/gemma.cpp/
!cmake -B build/
%cd build
!cmake --preset make
!cmake --build --preset make -j 2

In [ ]:
import kagglehub

# path = kagglehub.model_download("google/gemma/gemmaCpp/2b-it-sfp")
# print("Path to model files:", path)

path = kagglehub.model_download("google/gemma/gemmaCpp/7b-it-sfp")
print("Path to model files:", path)

In [ ]:
#%cd /root/.cache/kagglehub/models/google/gemma/gemmaCpp/2b-it-sfp/4
#!mv 2b-it-sfp.sbs /content/2b-it-sfp.sbs
#!mv tokenizer.spm /content/tokenizer.spm

%cd /root/.cache/kagglehub/models/google/gemma/gemmaCpp/7b-it-sfp/3
!mkdir /content/gemmacpp_7b_it_sfp
!mv 7b-it-sfp.sbs /content/gemmacpp_7b_it_sfp/7b-it-sfp.sbs
!mv tokenizer.spm /content/gemmacpp_7b_it_sfp/tokenizer.spm



%cd /content/gemma.cpp/build/

In [ ]:
!./gemma --tokenizer /content/tokenizer.spm --weights /content/2b-it-sfp.sbs --weight_type sfp --model 2b-it

#!./gemma --tokenizer /content/gemmacpp_7b_it_sfp/tokenizer.spm --weights /content/gemmacpp_7b_it_sfp/7b-it-sfp.sbs --weight_type sfp --model 7b-it